In [101]:
import Bio
import numpy as np
import math
import random
import matplotlib.pyplot as plt
random.seed()

In [73]:
genes = {"A":300,"B":300,"C":0,"D":0,"E":0,"F":0}
genarr = list(genes.values())
lam = 0.25
alph = 250
ki = 200
ka = 400
hi = 2
ha = 20

In [87]:
def compDs():
    #print(genes)
    dgAdt = (alph * (1/(1+np.power((genes["B"]/ki),hi)))) - lam*genes["A"]
    dgBdt = (alph * (1/(1+np.power((genes["A"]/ki),hi)))) - lam*genes["B"]
    dgCdt = (alph * (1/(1+np.power((ka/genes["A"]),ha))) * (1/(1+np.power((genes["D"]/ki),hi)))) - lam*genes["C"]
    dgDdt = (alph * (1/(1+np.power((ka/genes["A"]),ha))) * (1/(1+np.power((genes["C"]/ki),hi)))) - lam*genes["D"]
    dgEdt = (alph * (1/(1+np.power((ka/genes["B"]),ha))) * (1/(1+np.power((genes["F"]/ki),hi)))) - lam*genes["E"]
    dgFdt = (alph * (1/(1+np.power((ka/genes["B"]),ha))) * (1/(1+np.power((genes["E"]/ki),hi)))) - lam*genes["F"]
    return [dgAdt,dgBdt,dgCdt,dgDdt,dgEdt,dgFdt]

In [75]:
for i in compDs():
    print(i)
diffs = compDs()

{'A': 300, 'B': 300, 'C': 0, 'D': 0, 'E': 0, 'F': 0}
1.923076923076934
1.923076923076934
0.7902967861300224
0.7902967861300224
0.7902967861300224
0.7902967861300224
{'A': 300, 'B': 300, 'C': 0, 'D': 0, 'E': 0, 'F': 0}


In [76]:
deltt = 1
sigma = 10
tmax = 100
tmin = 0
y = 0
print(y)

0


In [91]:
sixdim = []
for rep in range(400):
    stop = random.randint(tmin,tmax-1)
    genes = {"A":300,"B":300,"C":0,"D":0,"E":0,"F":0}
    genarr = list(genes.values())
    for t in range(tmin,stop):
        for g in range(len(genes)):
            y = genarr[g] + (diffs[g]*deltt) + (sigma*np.sqrt(deltt)*random.gauss(0,1))
            genarr[g] = y
        #print(genes)
        genes = fixGenes(genes,genarr)
        diffs = compDs()
    #print(genes)
    genarr = list(genes.values())
    sixdim.append(genarr)

In [85]:
def fixGenes(genes, genarr):
    #print(genes)
    genes["A"] = genarr[0]
    genes["B"] = genarr[1]
    genes["C"] = genarr[2]
    genes["D"] = genarr[3]
    genes["E"] = genarr[4]
    genes["F"] = genarr[5]
    return genes

In [100]:
sixdim = np.array(sixdim)


[ 557.42779746  659.76426223  990.23933405   37.09177162  796.40439765
  399.34024972  335.56973531  948.41146964   46.40791012  318.18109133
  967.21106459  985.37530904  277.11655828  306.61976231   38.98631567
  334.70227133  400.89612391  240.94462479  341.47410656  979.2181156
   56.76406434  263.9777188   179.62770374   71.41497004  228.11753175
   64.51616109  309.45447572  967.47751711  920.17181653  678.61094592
  353.98023902  300.56998202  315.21650619   61.30069707   30.38339828
  321.39900563   46.75074507  353.63672443 1001.77351199  959.18300645
  746.0084553   819.04350078  844.75799475  306.45527165   47.20922913
  982.5606291   549.26071078  342.52303692  267.15283746  308.43349878
    7.45346091  967.85208243  943.5848873   892.47159272  774.31339874
  402.95894057  937.11079775  945.45458224   65.70189487   73.96710297
  444.75559983   44.89661028  932.36756261  378.88911073  538.23267278
  289.31147603  332.29664527  724.20027968  435.76184721   75.60290427
  404.5